In [42]:
import os
print(os.getcwd())

/home/st9540808


In [66]:
from bokeh.plotting import output_notebook
from caret_analyze import Architecture, Application, Lttng

from caret_analyze.plot import message_flow
from caret_analyze.plot import Plot

from caret_analyze.record import ResponseTime

output_notebook()

Loading BokehJS ...

In [48]:
trace_dir = './ntu_ws/evaluate/minimal_sub_20230418'

In [52]:
arch = Architecture('lttng', trace_dir)
arch.summary

683 events found.


loading: 100%|█████████████████████████████████████████████████████████| 683/683 [00:00<00:00, 509644.13it/s]


filtered to 417 events.


{'nodes': ('/caret_session_node', '/minimal_publisher', '/minimal_subscriber')}

In [53]:
paths = arch.search_paths('/minimal_publisher', '/minimal_subscriber')

type(paths) # list of multiple paths
arch.add_path('target_path', paths[0])
paths[0].summary.pprint() # shows nodes and topics in paths[0]

A search up to depth 15 has been completed. If the paths you want to measure cannot be found, consider specifying intermediate nodes. Also, if the number of paths is too large, consider filtering node/topic names. For details, see https://tier4.github.io/CARET_doc/latest/configuration/inter_node_data_path/.
path:
- message_context: null
  node: /minimal_publisher
- topic: /topic
- message_context: null
  node: /minimal_subscriber



In [64]:
arch = Architecture('yaml', './ntu_ws/evaluate/minimal_sub_20230418/architecture.yaml')
lttng = Lttng(trace_dir)
app = Application(arch, lttng)

path = app.get_path('target_path')
records = path.to_records()
response = ResponseTime(records)
response_records = response.to_response_records()
response_df = response_records.to_dataframe()
response_df

683 events found.


loading: 100%|██████████████████████████████████████████████████████████| 683/683 [00:00<00:00, 72133.50it/s]


,/topic/rclcpp_publish_timestamp/0_min,/topic/rclcpp_publish_timestamp/0_max,/minimal_subscriber/callback_0/callback_start_timestamp/0
0,1681806771711484635,1681806772211386207,1681806772211497128
1,1681806772211386207,1681806772711374939,1681806772711508112
2,1681806772711374939,1681806773211355571,1681806773211476363
3,1681806773211355571,1681806773711344969,1681806773711471254
4,1681806773711344969,1681806774211319385,1681806774211452899
5,1681806774211319385,1681806774711309337,1681806774711466570
6,1681806774711309337,1681806775211294822,1681806775211459253
7,1681806775211294822,1681806775711314402,1681806775711386304
8,1681806775711314402,1681806776211322359,1681806776211442959
9,1681806776211322359,1681806776711285490,1681806776711387370


In [67]:
plot = Plot.create_latency_timeseries_plot(app.communications)
latency_df = plot.to_dataframe()
latency_df

/minimal_publisher|/topic|/minimal_subscriber             
                   rclcpp_publish_timestamp [ns] latency [ms]
0                            1681806771711484635     0.084427
1                            1681806772211386207     0.110921
2                            1681806772711374939     0.133173
3                            1681806773211355571     0.120792
4                            1681806773711344969     0.126285
5                            1681806774211319385     0.133514
6                            1681806774711309337     0.157233
7                            1681806775211294822     0.164431
8                            1681806775711314402     0.071902
9                            1681806776211322359       0.1206
10                           1681806776711285490      0.10188
11                           1681806777211276312      0.08194
12                           1681806777711217596     0.083927

In [47]:
arch.export('./ntu_ws/evaluate/minimal_sub_20230418/architecture.yaml')

FileExistsError: [Errno 17] File exists: './ntu_ws/evaluate/minimal_sub_20230418/architecture.yaml'